<a href="https://colab.research.google.com/github/mori8/NLP-Pytorch-practice/blob/main/Chapter_2_NLP_%EA%B8%B0%EC%88%A0_%EB%B9%A0%EB%A5%B4%EA%B2%8C_%ED%9B%91%EC%96%B4%EB%B3%B4%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> 여기에 정리하지 않은 부분이 있다면 말로 설명하기 어려운 뭔가썸띵에 대한 주제라 생략한 것이므로 과거의 나를 욕하지 말고 책을 보자.

## NLP와 전산 언어학

- NLP의 목적: 정보 추출, 자동 음석 인식, 기계 번역, 감성 분석, 질의 응답, 요약 같은 실용적인 문제를 해결하는 방법을 개발하는 것
- 전산 언어학의 목적: 언어의 특징을 이해하는 방법을 개발하는 것

전산 언어학과 NLP는 긴밀한 관계에 있다. 전산 언어학에서 배운 언어적 지식을 NLP의 사전 지식으로 사용할 수 있으며, 반대로 NLP의 확률적인 방법과 머신러닝 방법을 적용해 전산 언허학이 찾으려는 답을 찾을 수 있다.

## 2.1 말뭉치, 토큰, 타입

- **말뭉치(corpus)**: 고전이나 현대의 모든 NLP 작업은 `말뭉치`라 부르는 텍스트 데이터로 시작한다. 말뭉치는 일반적으로 원시 텍스트(ASCII나 UIT-8 형태)와 이 텍스트에 연관된 메타데이터를 포함한다.
- **토큰(token)**: 토큰은 보통 의미를 가지는 단위로 선정되며, 영어에서 토큰은 공백 문자나 구두점으로 구분되는 단어와 숫자에 해당한다. 한국어, 터키어같은 교착어의 경우 공백이나 구두점으로 토큰이 분리되지 않는다. 원시 텍스트는 문자(바이트) 시퀀스지만, 일반적으로 문자를 토큰이라는 연속된 단위로 묶었을 때 유용하다. 텍스트를 토큰으로 나누는 과정을 **토큰화**라고 한다.
- **샘플(sample)** 혹은 **데이터 포인트(data point)**: 메타데이터(식별자, 레이블, 타입스탬프 등)를 갖는 텍스트다.
- **데이터셋(dataset)**: 샘플의 모음인 `말뭉치`를 데이터셋이라고 부른다.

In [1]:
import spacy
nlp = spacy.load('en')
text = "Mary, don't slap the green witch"
print([str(token) for token in nlp(text.lower())])

['mary', ',', 'do', "n't", 'slap', 'the', 'green', 'witch']


In [3]:
from nltk.tokenize import TweetTokenizer
tweet = u"Snow White and the Seven Degrees #MakeAMovieCold@midnight:-)"
tokenizer = TweetTokenizer()
print(tokenizer.tokenize(tweet.lower()))

['snow', 'white', 'and', 'the', 'seven', 'degrees', '#makeamoviecold', '@midnight', ':-)']


- **타입(type)**: 말뭉치에 등장하는 고유한 토큰이다. 말뭉치에 있는 모든 타입의 집합이 어휘 사전 또는 **어휘(lexicon)**다.
- 불용어(stopword): 자주 등장하지만 큰 의미가 없는 단어를 가리킨다. https://wikidocs.net/22530 참고

## 2.2 유니그램, 바이그램, 트라이그램, ..., n-그램

- **n-그램**: 텍스트에 있는 고정 길이(n)의 연속된 토큰 시퀀스다. 텍스트에서 n-그램을 만드는 일은 간단하며, `spaCy`나 `NLTK`같은 패키지에서도 n-그램을 편리하게 만드는 도구를 제공한다. 부분 단어(subword)가 유용한 정보를 전달하는 경우 n-그램을 유용하게 사용할 수 있다.

In [4]:
def n_grams(text, n):
  return [text[i:i + n] for i in range(len(text) - n + 1)]

cleaned = ['mary', ',', "n't", 'slap', 'green', 'witch', '.']
print(n_grams(cleaned, 3))

[['mary', ',', "n't"], [',', "n't", 'slap'], ["n't", 'slap', 'green'], ['slap', 'green', 'witch'], ['green', 'witch', '.']]


## 2.3 표제어와 어간

- **표제어(lemma)**: 단어의 기본형이다. 쉽게 생각하면, 사전에 등재된 단어다. 예를 들어 동사 'fly'를 생각해볼 때, flow, flew, flies, flown, flowing 등 어미가 바뀌며 여러 단어로 변형되는데, fly는 이런 모든 단어의 `표제어`다.
- **표제어 추출(lemmatization)**: 토큰을 표제어로 바꾸어 벡터 표현의 차원을 줄이는 방법이다.
- **어간 추출(stemming)**: 표제어 추출 대신에 사용하는 축소 기법으로, 수동으로 만든 규칙을 사용하여 단어의 끝을 잘라 어간이라는 공통 형태로 축소한다.

In [5]:
import spacy
nlp = spacy.load('en')
doc = nlp(u"he was running late")
for token in doc:
  print('{} -> {}'.format(token, token.lemma_))

he -> -PRON-
was -> be
running -> run
late -> late


## 2.5 단어 분류하기: 품사 태깅

문서에 레이블을 할당하는 개념을 단어나 토큰으로 확장할 수 있다.

In [6]:
import spacy
nlp = spacy.load('en')
doc = nlp(u"Mary slapped the green witch.")
for token in doc:
  print('{} - {}'.format(token, token.pos_))

Mary - PROPN
slapped - VERB
the - DET
green - ADJ
witch - NOUN
. - PUNCT


## 2.6 청크 나누기와 개체명 인식

연속된 여러 토큰으로 구분되는 텍스트 구에 레이블을 할당하는 것을 `청크 나누기` 혹은 `부분 구문 분석`이라고 한다. 부분 구문 분석의 목적은 명사, 동사, 형용사 같은 문법 요소로 구성된 고차원의 단위를 유도해내는 것이다. 부분 구문 분석 모델 훈련에 사용할 데이터가 없다면 품사 태깅에 정규식을 적용해 부분 구문 분석을 근사할 수 있다. 영어 등 널리 사용하는 언어는 `spaCy` 등에 포함된 모델을 사용해 부분 구문 분석을 수행할 수 있다.

In [1]:
import spacy
nlp = spacy.load('en')
doc = nlp(u"Mary slapped the green witch.")
for chunk in doc.noun_chunks:
  print('{} - {}'.format(chunk, chunk.label_))

Mary - NP
the green witch - NP


## 2.7 문장 구조

- **구성 구문 분석(constituent parsing)**
- **의존 구문 분석(dependency parsing)**

## 2.8 단어 의미와 의미론

- [WordNet](https://wordnet.princeton.edu): 모든 영어 단어의 관계와 의미를 수집하는 프로젝트
- 단어의 의미는 문맥을 통해 결정되기도 하므로, 텍스트에서 단어의 의미를 자동으로 찾는 일은 NLP에 적용된 첫 번째 준지도 학습이다.